# サンプルエージェントの作成

## (事前準備)ドキュメントへのリンクの有効化

このサンプルノートブックでドキュメントのリンクを有効化するためには, このファイルがあるディレクトリのサーバをlocalhostの5500番ポートに公開しておくこと.

例えば, Pythonを使って

```bash
$ python -m http.server 5500
```
としてサーバを起動できる.

あるいは, VSCodeを使用している場合は, 拡張機能[LiveServer](https://marketplace.visualstudio.com/items?itemName=ritwickdey.LiveServer)をインストールし, このファイルがあるディレクトリを開いた状態で画面右下の「Go Live」アイコンをクリックすれば自動的に5500番ポートにサーバが起動する.

## 概要

行動判断を行う簡単なエージェントの作成方法を説明する. `README.md`にあるように, シミュレータを実行するときに各エージェントがインポートされる際`getUserAgentClass`が呼ばれ, 実装したエージェントがシミュレータ内で返される. ここではPythonで自作したエージェントを実際にシミュレータ上で動かすまでを確認する. 
ドキュメントの[「シミュレーションに関する説明/シミュレーションの処理の流れについて#シミュレーションの処理の流れ」](http://localhost:5500/docs/html/core/a198011.html#section_simulation_flow)
に実装すべきメソッドとともに記述されているので, よく確認しておくとよい. 大雑把に`(validate)->perceive->makeObs->deploy->control->behave->...`の順に実行されていく. 以下で, クラス定義からそれぞれのメソッドを説明しつつ実装していき, 最終的に初期行動判断モデルと対戦させて結果を確認する.

なお,完全な形のエージェントの実装例は[simulator/sample/modules/R7ContestSample/R7ContestSample/R7ContestPyAgentSamle01.py](http://localhost:5500/docs/html/core/a198025.html)にあるので,それも併せて参照されたい.

## クラスの定義

エージェントのインスタンスを作成するための抽象クラスを`Agent`を継承して定義する. ここでは`SampleAgent`という名前のクラスにする. エージェントの形として1機につき1つのエージェントを割り当てる分散方式と, 編隊全体で1つのエージェントを割り当てる中央集権方式がある. 分散方式の場合は`Agent`の代わりに`SingleAssetAgent`を,継承してもよい.

`Agent`と`SingleAssetAgent`の違いは,割り当てられた各機の情報にアクセスするメンバ変数の違いのみである. `Agent`はdict型のメンバ変数`parents`を使用し, `SingleAssetAgent`は`parent`を使用する想定である. ただし, `SingleAssetAgent`でも`parents`を用いたアクセスは可能である.

In [ ]:
# 分散方式の場合

from ASRCAISim1.core import SingleAssetAgent # type: ignore

class SampleAgent(SingleAssetAgent): # 中央集権方式と同様にAgentを継承してもよい
    ...

In [ ]:
# 中央集権方式の場合

from ASRCAISim1.core import Agent

class SampleAgent(Agent):
    ...

## `initialize`メソッドの実装

ユーザによる初期化処理は原則として, コンストラクタではなくinitializeメソッドで行うこととしている.

コンストラクタでは,`modelConfig`と`instanceConfig`が渡されており,それぞれ同名のメンバ変数として保持されているので,初期化処理はそれを参照して行う.

本シミュレータではモデルのパラメータセットを`modelConfig`という名称の`json`型変数で, インスタンス固有のパラメータセットを`instanceConfig`という名称の`json`型変数で取り扱うこととしている. なお, `modelConfig`とはFactoryに登録するためのエージェントに関する設定で, 

```json
{
    "Factory":{
        "Agent":{
            "modelName":{
                "class":"className",
                "config":{...}
            }
        }
    }
}
```

の"config"の部分に記載される{...}のdictが該当する. 例えば1[tick]ごとではない処理周期としたい場合などには,ドキュメントの[「シミュレーションに関する説明/シミュレーション登場物の種類について/Entityの基本仕様#処理周期の指定」](http://localhost:5500/docs/html/core/a198006.html#section_simulation_entity_period)に従い, `modelConfig`に処理周期に関する記述を追加する.

```json
{
    "interval":{
        "unit":"time",
        "step":3.0,
        "perceive":1.0,
        "control":1.0,
        "behave":1.0
    },
    ...
}
```

上記の場合, "unit"を"time"で"step"を3.0にしているので, Agentの行動判断周期は3.0[s], "perceive", "control", "behave"の間隔は1.0[s]となる. こちらはjsonファイルとして保存しておき, シミュレーション実行時に`getUserAgentModelConfig`関数を呼んだときに読み込んで返す想定. なお, `modelConfig`を参照したい場合は`[]`でkeyを指定すれば`nljson`型で得られるので, Pythonプリミティブ型として扱う場合は`()`を付ける.

```Python
self.time_params = self.modelConfig['interval']()
```

また, ドキュメントの[「シミュレーションに関する説明/jsonによる確率的パラメータ設定」](http://localhost:5500/docs/html/core/a198012.html)にあるユーティリティを用いて確率的な選択やデフォルト値の設定も可能. その場合の出力はPythonプリミティブ型となるため`()`の付加は不要. 現バージョンでは乱数生成器には std::mt19937 しか使用できないが, `self.randomGen`として基底クラスで予め生成されているためこれを使用する.

```Python
self.withRandom = getValueFromJsonKR(self.modelConfig,"R",self.randomGen)
```

以下は中央集権方式の場合の実装. `modelConfig`から設定した"interval"を参照してattributeとして`time_params`と自軍が"Red"か"Blue"のどちらかを示す`own`と機体に対するコマンド([deployメソッドの実装](#deployメソッドの実装)で後述)をまとめた`actionInfos`を加えている. 実際にはその後の`makeObs`などにおいて参照したい情報をあらかじめ持っておくとよい.

In [ ]:
class SampleAgent(Agent):

    def initialize(self):
        super().initialize() # 基底クラスのinitializeも必ず呼び出すこと。
        self.time_params = self.modelConfig['interval']()
        self.own = self.getTeam()
        self.actionInfos=[self.ActionInfo() for _ in self.parents] # ActionInfoについては後述

## `validate`メソッドの実装

modelConfigとして設定可能とするパラメータを選択し, 他の登場物に依存する初期化処理が必要な場合に使うメソッド. rulerやparentのobservablesに依存するものがあるような場合を想定している. 以下では戦域中心から場外ラインまでの距離, 戦域中心から防衛ラインまでの距離, [陣営座標系](http://localhost:5500/docs/html/BasicAgentUtility/a200118.html)(進行方向が+x方向となるようにz軸まわりに回転させ、防衛ライン中央が原点となるように平行移動させた座標系)の変換クラスのインスタンスを取得している. このメソッドは必須ではない.

In [ ]:
from BasicAgentUtility.util import TeamOrigin # 陣営座標系を表すユーティリティクラス

def validate(self):
    #Rulerに関する情報の取得
    rulerObs=self.manager.getRuler()().observables()
    self.dOut=rulerObs["dOut"] # 戦域中心から場外ラインまでの距離
    self.dLine=rulerObs["dLine"] # 戦域中心から防衛ラインまでの距離
    self.teamOrigin=TeamOrigin(self.own==rulerObs["eastSider"],self.dLine) # 陣営座標系変換クラス定義

## `makeObs`メソッドの実装

observables(シミュレーション中に観測可能な情報)からObservation(行動判断を行うための情報に変換されたもの)を生成する関数. observablesとして取得可能な情報の一覧はドキュメントの[「第4回空戦AIチャレンジ向けチュートリアル/Agent が入出力 observables と commands の形式#Agent が受け取ることのできる observables」](http://localhost:5500/docs/html/core/a198024.html#section_r7_contest_agent_observables)に記載されている.

なお, 上記のページには書かれていないが、記載されているobservablesは生存中のparentからしか得られない. 生存していないparentについては生存していないことを表す
```python
{
    "isAlive":  False, # 生存中か否か
}
```
のみとなるが,その誘導弾情報については,生存中の他の`parent`の`observables`経由で以下のように最新の情報を引き続き得ることができる.
```python
# 生存中のparentを探す
for port,parent in self.parents.items():
    if parent.isAlive():
        firstAlive=parent
        break

# 以下は生存中のparentのobservablesから'DeadParent'というfull nameを持つparentの誘導弾情報を取得する場合の例
# ここで, at_pはnljson型のオブジェクトに対してjson pointer形式でネストされた場所にある値を取得できる関数である.
# 同様に, キーの存在チェックをjson pointer形式で行うcontains_p関数も提供されている.
deadObs = firstAlive.observables.at_p('/shared/fighter').at('DeadParent')

missiles = deadObs.at_p('/weapon/missiles') # 各誘導弾のobservablesがリストで得られる

for m in missiles:
    ... # 取得した誘導弾情報を処理する
```

### 味方機の運動状態の取得

味方機の運動状態を取得したい場合は,以下のようにしてjsonから`MotionState`オブジェクトを復元するとよい. `MotionState`オブジェクトを用いると, 位置や速度等をメンバ変数として取得できるほか, 様々な座標変換や外挿等の機能が使用できるようになる. `MotionState`クラスの説明は[こちら](http://localhost:5500/docs/html/core/a198013.html)に記載されている.

```python
from ASRCAISim1.core import MotionState

self.ourMotion = []
for port, parent in self.parents.items():
    if parent.isAlive():
        MotionState(parent.observables['motion']).transformTo(self.getLocalCRS())
    else:
		self.ourMotion.append(MotionState())
```

なお, `MotionState.transformTo(self.getLocalCRS())`は, Agent自身の行動判断用の座標系での運動状態に変換する関数である. デフォルトではRulerのlocalCRSが使用される. 本コンペティションの設定においてはシミュレータ本体の基準座標系と同じものとなり, [NED(North-East-Down)の地球の曲率を無視した座標系](http://localhost:5500/docs/html/core/a198004.html#section_simulation_Coordinate_PureFlatCRS)となるので, 必ずしもこの変換を行わなくとも支障はない.


### 彼機の航跡の取得

彼機の航跡を取得したい場合は,以下のようにしてjsonから`Track3D`オブジェクトを復元するとよい. `Track3D`オブジェクトを用いると, 位置や速度等をメンバ変数として取得できるほか, 座標変換や外挿等の機能が使用できるようになる. また、parentへの射撃目標の指示も`Track3D`を与えることで行う必要がある. `Track3D`クラスの説明は[こちら](http://localhost:5500/docs/html/core/a198016.html#section_simulation_Track_Track3D)に記載されている. また,各機のレーダが観測した彼機の航跡は,全てマージされて各parentのobservablesの`/sensor/track`に格納されている. そのため,特段の必要が無い限り,各機のレーダ航跡(`/sensor/radar/track`)を個別に取得する必要はない.

```Python
# まずは生存中のparentを探す
firstAlive = None
for port,parent in self.parents.items():
    if parent.isAlive():
        firstAlive = parent
        break

# マージされた航跡を取得し,Track3Dオブジェクトとして復元
# MotionStateと同様に, localCRS上の値に変換しておくことを推奨する.
self.lastTrackInfo = [Track3D(t).transformTo(self.getLocalCRS()) for t in firstAlive.observables.at_p('/sensor/track')]

# 味方機との距離が最も近い順にソートするユーティリティ関数がBasicAgentUtilityプラグインに実装されているので,必要に応じ使用すること.
from BasicAgentUtility.util import sortTrack3DByDistance
sortTrack3DByDistance(self.lastTrackInfo,self.ourMotion,True)
```

### 味方の誘導弾情報の取得

味方の誘導弾のobservablesの形式は,ドキュメントの[「第4回空戦AIチャレンジ向けチュートリアル/Agent が入出力 observables と commands の形式#誘導弾に関するobservables」](http://localhost:5500/docs/html/core/a198024.html#section_r7_contest_missile_observables)
に記載されている.

例えば,射撃時刻の古い順にソートする場合は以下のようにすればよい.

```python
from ASRCAISim1.core import Time, TimeSystem

def launchedT(m):
    # Timeオブジェクトは比較演算子が実装されているので,
    # 飛翔中(isAliveとhasLaunchedが両方True)の場合はlaunchedT,
    # そうでなければnp.infのTimeオブジェクトをキーとして返すことで古い順にソートできる
	return Time(m["launchedT"]) if m["isAlive"] and m["hasLaunched"] else Time(np.inf,TimeSystem.TT)
	self.msls=sorted(sum([[m for m in f.at_p("/weapon/missiles")] for f in self.ourObservables],[]),key=launchedT)
```

また, 誘導弾情報も 運動状態や目標航跡の情報を`MotionState`や`Track3D`で持っているため, 以下のような形でこれらのオブジェクトを復元して用いるとよい.

```python
for mObs in self.msls:
    motion=MotionState(mObs["motion"]).transformTo(self.getLocalCRS())
    target=Track3D(mObs["target"]).transformTo(self.getLocalCRS())
    ...
```

### 彼側の誘導弾情報の取得

各parentが検出した彼側の誘導弾のリストは, 各parentのobservablesの`sensor/mws/track`に格納されている.
以下のようにしてjsonから`Track2D`オブジェクトを復元するとよい.
`Track2D`オブジェクトを用いると, 位置や速度等をメンバ変数として取得できるほか, 座標変換や外挿等の機能が使用できるようになる.
`Track2D`クラスの説明は[こちら](http://localhost:5500/docs/html/core/a198016.html#section_simulation_Track_Track2D)に記載されている.

```python
self.mws={}
for port, parent in self.parents.items():
    self.mws[parent.getFullName()]=[]
    if parent.isAlive():
		if parent.observables.contains_p("/sensor/mws/track"):
            for mObs in parent.observables.at_p("/sensor/mws/track"):
                # MotionStateやTrack3Dと同様, localCRS上の値に変換しておくことを推奨する.
                self.mws[parent.getFullName()].append(
                    Track2D(mObs).transformTo(self.getLocalCRS())
                )
        motion=MotionState(parent.observables['motion']).transformTo(self.getLocalCRS())
        # 特定の方向に近い順にソートするユーティリティ関数がBasicAgentUtilityプラグインに実装されているので,必要に応じ使用すること.
        # 以下は自身の正面(1,0,0)に近い順いソートする例.
		sortTrack2DByAngle(self.mws[parent.getFullName()],motion,np.array([1,0,0]),True)

```

### 実装例

以下では特に何も加工しないが, ログとして残す情報をobservablesから選定して返している. ログとして残すにはjson形式で保存できなければいけないため, Pythonプリミティブな型としてjsonとして保存できる形で扱っている(`numpy.ndarray`の場合は自動的に`list`に変換するようになっているため, 保存可能.). また, メモリ制限があるので, 適宜調整されたい.

上記の情報を抽出してObservationとして加工する具体例が[simulator/sample/modules/R7ContestSample/R7ContestSample/R7ContestPyAgentSamle01.py](http://localhost:5500/docs/html/core/a198025.html)にあるので, そちらも参照されたい. このメソッドは必須となる.

In [ ]:
import numpy as np
from ASRCAISim1.core import MotionState, Track3D, Track2D, Time, TimeSystem, nljson
from BasicAgentUtility.util import sortTrack3DByDistance, sortTrack2DByAngle

def makeObs(self):
    #味方機(parents→parents以外の順)
    self.ourMotion=[]
    self.ourObservables=[]
    firstAlive=None
    for port,parent in self.parents.items():
        if parent.isAlive():
            firstAlive=parent
            break

    parentFullNames=set()
    # まずはparents
    for port, parent in self.parents.items():
        parentFullNames.add(parent.getFullName())
        if parent.isAlive():
            self.ourMotion.append(MotionState(parent.observables["motion"]).transformTo(self.getLocalCRS()))
            #残存していればobservablesそのもの
            self.ourObservables.append(parent.observables)
        else:
            self.ourMotion.append(MotionState())
            #被撃墜or墜落済なら本体の更新は止まっているので残存している親が代理更新したものを取得(誘導弾情報のため)
            self.ourObservables.append(
                firstAlive.observables.at_p("/shared/fighter").at(parent.getFullName()))

    # その後にparents以外
    for fullName,fObs in firstAlive.observables.at_p("/shared/fighter").items():
        if not fullName in parentFullNames:
            if fObs.at("isAlive"):
                self.ourMotion.append(MotionState(fObs["motion"]).transformTo(self.getLocalCRS()))
            else:
                self.ourMotion.append(MotionState())

            self.ourObservables.append(fObs)

    #彼機(味方の誰かが探知しているもののみ)
    #観測されている航跡を、自陣営の機体に近いものから順にソートしてlastTrackInfoに格納する。
    #lastTrackInfoは行動のdeployでも射撃対象の指定のために参照する。
    firstAlive=None
    for port,parent in self.parents.items():
        if parent.isAlive():
            firstAlive=parent
            break

    self.lastTrackInfo=[Track3D(t).transformTo(self.getLocalCRS()) for t in firstAlive.observables.at_p("/sensor/track")]
    sortTrack3DByDistance(self.lastTrackInfo,self.ourMotion,True)

    #味方誘導弾(射撃時刻の古い順にソート)
    def launchedT(m):
        return Time(m["launchedT"]) if m["isAlive"] and m["hasLaunched"] else Time(np.inf,TimeSystem.TT)
    self.msls=sorted(sum([[m for m in f.at_p("/weapon/missiles")] for f in self.ourObservables],[]),key=launchedT)

    #彼側誘導弾(各機の正面に近い順にソート)
    self.mws=[]
    for fIdx,fMotion in enumerate(self.ourMotion):
        fObs=self.ourObservables[fIdx]
        self.mws.append([])
        if fObs["isAlive"]:
            if fObs.contains_p("/sensor/mws/track"):
                for mObs in fObs.at_p("/sensor/mws/track"):
                    self.mws[fIdx].append(Track2D(mObs).transformTo(self.getLocalCRS()))
            sortTrack2DByAngle(self.mws[fIdx],fMotion,np.array([1,0,0]),True)

    #必要なものだけを返す.
    #json.dumpでjson化できる形でなければならないのでTrack3D等はそのままでは返せない.
    #以下のようにメンバ変数を個別に選んでもよいし、nljson型を挟んでもよい.

    #lastTrackInfo = [{'truth':track.truth.__str__(), 'time': track.time, 'pos': list(track.pos), 'vel':list(track.vel)} for track in self.lastTrackInfo]
    #msls = [m() for m in self.msls]
    #return {'ourMotion':self.ourMotion, 'lastTrackInfo': lastTrackInfo, 'msl': msls}

    return nljson({
        'ourMotion':self.ourMotion,
        'lastTrackInfo':self.lastTrackInfo,
        'msls':self.msls
    })() #nljsonを生成してから()でプリミティブ型に変えるのでも可

## `observation_space`メソッドの実装

状態空間を定義するメソッド. `makeObs`メソッドで返すObservationの形を定義する. 強化学習を実装する際は`makeObs`で返すObservationの型と一致させる必要がある. このメソッドは必須となる. 以下ではObservationの形式に従わず適当なダミーデータを返している. 

In [ ]:
from gymnasium import spaces

def observation_space(self):
    return spaces.Box(low=0.0,high=1.0,shape=(1,))

## `action_space`メソッドの実装

生成したObservationをもとに作成する行動の空間を定義するメソッド. `Agents/MyAgent/__init__.py`で定義した`policy`により行動を返す想定. 今回は制限時間以内に何らかの行動を返さなければここで定義された行動空間からランダムサンプリングされる. 以下では例として`spaces.Discrete`により0,1の整数値を定義するような実装としている. このメソッドは必須となる.

In [ ]:
from gymnasium import spaces

def action_space(self):
    return spaces.Discrete(2)

独自の実装としたい場合は`gymnasium.spaces.Space`を継承して`sample`メソッドをオーバーライドする.

In [ ]:
from gymnasium import spaces
from random import randint

class ActionSpace(spaces.Space):
    def sample(self):
        return randint(0,1)

`Agents/MyAgent/__init__.py`で観測されている航跡の中で自陣営の機体に最も近いものの速さを返す`policy`を定義する(あくまで一例で, Observationから後に続くdepolyメソッドに渡すactionとして適切なものを実装する想定).

In [ ]:
import numpy as np
from ASRCAISim1.policy import StandalonePolicy

class DummyPolicy(StandalonePolicy):
    def step(self,observation,reward,done,info,agentFullName,observation_space,action_space):
        from ASRCAISim1.core import MotionState
        if len(observation['lastTrackInfo']) > 0:
            vel = np.linalg.norm(observation['lastTrackInfo'][0]['vel'])
        else:
            # 何も観測されていない場合は最初のparentの速度を使用
            vel = np.linalg.norm(MotionState(observation['ourMotion'][0]).vel())
        return vel

## `deploy`メソッドの実装

なんらかの`policy`により`observation`から生成した`action`に対して`decision`や`commands`により行動を行うメソッド. このメソッドは1stepに1回実行される. `decision`はAgentクラス間に共通の行動意図を表現するためものであり,ドキュメントの[「行動判断モデルの学習のための機能/模倣学習のための機能#Agent クラス間に共通の行動意図表現について」](http://localhost:5500/docs/html/core/a198021.html#section_training_imitation_common_decision_format)に説明がある. 

`decision`は模倣学習の教師役として使用しない場合は省略してもよい.


記述する`commands`の詳細はドキュメントの[「第4回空戦AIチャレンジ向けチュートリアル/Agent が入出力 observables と commands の形式#Agent が出力すべき commands」](http://localhost:5500/docs/html/core/a198024.html#section_r7_contest_agent_commands)に記載されている.

observablesやObservationに応じてどのように動いてどのタイミングで射撃するかなどを決めるロジックを考案して実装する.
このとき,以下のような形で行動をcommandsに変換するためのデータクラスを用意しておくと加工や再利用が容易になる.

```python
import numpy as np
from ASRCAISim1.core import Track3D

class ActionInfo():
     def __init__(self):
        self.dstDir=np.array([1.0,0.0,0.0]) #目標進行方向
        self.dstAlt=10000.0 #目標高度
        self.velRecovery=False #下限速度制限からの回復中かどうか
        self.asThrottle=False #加減速についてスロットルでコマンドを生成するかどうか
        self.keepVel=False #加減速について等速(dstAccel=0)としてコマンドを生成するかどうか
        self.dstThrottle=1.0 #目標スロットル
        self.dstV=300 #目標速度
        self.launchFlag=False #射撃するかどうか
        self.target=Track3D() #射撃対象
        self.lastShotTimes={} #各Trackに対する直前の射撃時刻
```

例えば,

action, obsevables, Observationを加工, 必要な情報を取得->`actionInfos`をそれぞれの戦闘機で更新->`commands`を指定

という流れで実装するとよい.

以下では`action`は観測されている航跡の中で自陣営の機体に最も近いものの速さを表すとして, 彼機との距離に応じて接近したり遠ざかったりし, 地面に激突することを避けるために一定の高度になったら上昇, 一定の距離に達したら射撃するといったシンプルなロジックを実装している.

このメソッドは必須.

In [ ]:
def deploy(self,action):
    for pIdx,parent in enumerate(self.parents.values()):
        if(not parent.isAlive()):
            continue

        self.observables[parent.getFullName()]["decision"]={
            "Roll":["Don’t care"],"Horizontal":["Az_BODY",0.0],
            "Vertical":["El",100.0],
            "Throttle":["Vel",150.0],
            "Fire":[False,Track3D().to_json()]
        }

    # observablesから必要な情報を取得
    tick = self.manager.getTickCount() # シミュレーション開始時からの経過tick数を取得
    time = self.manager.getTime() # Timeオブジェクトとしてシミュレーション中の時刻を取得
    elapsedTime = self.manager.getElapsedTime() # シミュレーション開始時からの経過秒数を取得
    dist = {}
    for pIdx, parent in enumerate(self.parents.values()):
        if(parent.isAlive()):
            myMotion=MotionState(parent.observables["motion"]).transformTo(self.getLocalCRS())
            tracks = []
            for t in parent.observables.at_p("/sensor/track"):
                ret=-1.0
                track = Track3D(t).transformTo(self.getLocalCRS())
                tmp=np.linalg.norm(track.pos()-myMotion.pos())
                if(ret<0 or tmp<ret):
                    ret = tmp
                tracks.append({'track':track, 'distance':ret})
            dist[pIdx] = tracks

    # actionInfosの更新
    for pIdx,parent in enumerate(self.parents.values()):
        if(not parent.isAlive()):
            continue
        # 自陣営の機体に最も近いものの速さを自分の速さに設定する
        self.actionInfos[pIdx].dstV = action
        
        # 自分に最も近い機体との距離が20000以下の場合は逃げるために距離をとり, それ以外の場合は接近する
        # 何も観測できていない場合は前回の値のまま変更しない
        if len(dist[pIdx]) > 0:
            myMotion=MotionState(parent.observables["motion"]).transformTo(self.getLocalCRS())
            d = sorted(dist[pIdx], key=lambda x: x['distance'])[0]
            diff=myMotion.pos()-d['track'].pos()
            if d['distance'] <= 20000:
                print('escape')
                self.actionInfos[pIdx].dstDir=diff/np.linalg.norm(diff)
            else:
                print('move forward')
                self.actionInfos[pIdx].dstDir=-diff/np.linalg.norm(diff)

        # 目標進行方向のz成分が正で高度が5000以下の場合は上昇
        if self.actionInfos[pIdx].dstDir[2]>0 and np.array(myMotion.pos())[2]>=-5000:
            print('up')
            self.actionInfos[pIdx].dstDir[2] = -self.actionInfos[pIdx].dstDir[2]

        # 自分に最も近い機体との距離が20000以上22000以下の場合は対象に対して射撃する
        # 何も観測できていない場合は何もしない
        if len(dist[pIdx]) > 0:
            self.actionInfos[pIdx].target = d['track']
            if d['distance'] >= 20000 and d['distance']<=22000:
                print('launch')
                self.actionInfos[pIdx].launchFlag=True
            else:
                self.actionInfos[pIdx].launchFlag=False
        else:
            self.actionInfos[pIdx].launchFlag=False


    # commandsに設定
    # これまでの演算をlocalCRS上で行ってきたが,
    # commandsでparentに渡すときはparentの座標系に戻さなければならない.
    for pIdx,parent in enumerate(self.parents.values()):
        if(not parent.isAlive()):
            continue
        originalMyMotion=MotionState(parent.observables["motion"])
        myMotion=MotionState(parent.observables["motion"]).transformTo(self.getLocalCRS())
        print(pIdx, myMotion.pos(), self.actionInfos[pIdx].dstV, self.actionInfos[pIdx].dstDir)

        #元のparent座標系に戻す
        cmdDstDir=originalMyMotion.dirAtoP(self.actionInfos[pIdx].dstDir,myMotion.pos(),self.getLocalCRS())

        self.commands[parent.getFullName()] = {
            "motion": { #機動の指定。以下の指定方法は一例。
                "dstDir": cmdDstDir, #進みたい方向を指定
                "dstV": self.actionInfos[pIdx].dstV #進みたい速度を指定
            },
            "weapon": {
                "launch": self.actionInfos[pIdx].launchFlag, #射撃要否を bool で指定
                "target": self.actionInfos[pIdx].target #射撃目標の Track3D を 指定
            }
        }

## `control`メソッドの実装

より細かく1tick単位の処理を行いたいときに実装するメソッド. `decision` and/or `commands`の複雑な生成処理を行う場合等に用いる. `commands`は`deploy`メソッドで計算してもよいが, ここでより高頻度に計算してもよい(どちらかで実装する必要はある). このメソッドは必須ではない.

## `behave`メソッドの実装

`control`メソッドの次に呼ばれる. `control`メソッドと同様により細かく1tick単位の処理を行いたいときに実装するメソッド. `decision` and/or `commands`の複雑な生成処理を行う場合等に用いる. このメソッドは必須ではない.

## `perceive`メソッドの実装

`behave`メソッドの次に呼ばれる. `control`メソッドと同様により細かく1tick単位の処理を行いたいときに実装するメソッド. `decision` and/or `commands`の複雑な生成処理を行う場合等に用いる. このメソッドは必須ではない.

## (参考)`serializeInternalState`メソッドについて

完全な形のエージェントの実装例[simulator/sample/modules/R7ContestSample/R7ContestSample/R7ContestPyAgentSamle01.py](http://localhost:5500/docs/html/core/a198025.html)には、`serializeInternalState`というメソッドが実装されている.

これは,ドキュメントの[「シミュレーションに関する説明/オブジェクトのシリアライゼーション#内部状態のシリアライゼーション (Experimental)」](http://localhost:5500/docs/html/core/a198014.html#autotoc_md7)に記載されている,シミュレーションの内部状態をjson等にシリアライズする実験的機能のためのメソッドである.

この機能が不要であれば,`serializeInternalState`というメソッドは実装しなくても全く問題はない.

## 全てをまとめる

今まで実装したメソッドをまとめて以下のように`MyAgent.py`として作成して, `Agents/MyAgent`以下に保存する. なお, 中央集権方式をとっている.

In [ ]:
import numpy as np
from gymnasium import spaces
from ASRCAISim1.core import (
    Agent,
    MotionState,
    Track3D,
    Track2D,
    Time,
    TimeSystem,
    nljson
)
from BasicAgentUtility.util import (
    sortTrack3DByDistance,
    sortTrack2DByAngle,
    TeamOrigin
)

class SampleAgent(Agent):

    class ActionInfo():
        def __init__(self):
            self.dstDir=np.array([1.0,0.0,0.0]) #目標進行方向
            self.dstAlt=10000.0 #目標高度
            self.velRecovery=False #下限速度制限からの回復中かどうか
            self.asThrottle=False #加減速についてスロットルでコマンドを生成するかどうか
            self.keepVel=False #加減速について等速(dstAccel=0)としてコマンドを生成するかどうか
            self.dstThrottle=1.0 #目標スロットル
            self.dstV=300 #目標速度
            self.launchFlag=False #射撃するかどうか
            self.target=Track3D() #射撃対象
            self.lastShotTimes={} #各Trackに対する直前の射撃時刻


    def initialize(self):
        super().initialize() # 基底クラスのinitializeも必ず呼び出すこと。
        self.time_params = self.modelConfig['interval']()
        self.own = self.getTeam()
        self.actionInfos=[self.ActionInfo() for _ in self.parents]


    def observation_space(self):
        return spaces.Box(low=0.0,high=1.0,shape=(1,))


    def action_space(self):
        return spaces.Box(low=150.0, high=450.0, shape=(1,))


    def validate(self):
        #Rulerに関する情報の取得
        rulerObs=self.manager.getRuler()().observables()
        self.dOut=rulerObs["dOut"] # 戦域中心から場外ラインまでの距離
        self.dLine=rulerObs["dLine"] # 戦域中心から防衛ラインまでの距離
        self.teamOrigin=TeamOrigin(self.own==rulerObs["eastSider"],self.dLine) # 陣営座標系変換クラス定義


    def makeObs(self):
        #味方機(parents→parents以外の順)
        self.ourMotion=[]
        self.ourObservables=[]
        firstAlive=None
        for port,parent in self.parents.items():
            if parent.isAlive():
                firstAlive=parent
                break

        parentFullNames=set()
        # まずはparents
        for port, parent in self.parents.items():
            parentFullNames.add(parent.getFullName())
            if parent.isAlive():
                self.ourMotion.append(MotionState(parent.observables["motion"]).transformTo(self.getLocalCRS()))
                #残存していればobservablesそのもの
                self.ourObservables.append(parent.observables)
            else:
                self.ourMotion.append(MotionState())
                #被撃墜or墜落済なら本体の更新は止まっているので残存している親が代理更新したものを取得(誘導弾情報のため)
                self.ourObservables.append(
                    firstAlive.observables.at_p("/shared/fighter").at(parent.getFullName()))

        # その後にparents以外
        for fullName,fObs in firstAlive.observables.at_p("/shared/fighter").items():
            if not fullName in parentFullNames:
                if fObs.at("isAlive"):
                    self.ourMotion.append(MotionState(fObs["motion"]).transformTo(self.getLocalCRS()))
                else:
                    self.ourMotion.append(MotionState())

                self.ourObservables.append(fObs)

        #彼機(味方の誰かが探知しているもののみ)
        #観測されている航跡を、自陣営の機体に近いものから順にソートしてlastTrackInfoに格納する。
        #lastTrackInfoは行動のdeployでも射撃対象の指定のために参照する。
        firstAlive=None
        for port,parent in self.parents.items():
            if parent.isAlive():
                firstAlive=parent
                break

        self.lastTrackInfo=[Track3D(t).transformTo(self.getLocalCRS()) for t in firstAlive.observables.at_p("/sensor/track")]
        sortTrack3DByDistance(self.lastTrackInfo,self.ourMotion,True)

        #味方誘導弾(射撃時刻の古い順にソート)
        def launchedT(m):
            return Time(m["launchedT"]) if m["isAlive"] and m["hasLaunched"] else Time(np.inf,TimeSystem.TT)
        self.msls=sorted(sum([[m for m in f.at_p("/weapon/missiles")] for f in self.ourObservables],[]),key=launchedT)

        #彼側誘導弾(各機の正面に近い順にソート)
        self.mws=[]
        for fIdx,fMotion in enumerate(self.ourMotion):
            fObs=self.ourObservables[fIdx]
            self.mws.append([])
            if fObs["isAlive"]:
                if fObs.contains_p("/sensor/mws/track"):
                    for mObs in fObs.at_p("/sensor/mws/track"):
                        self.mws[fIdx].append(Track2D(mObs).transformTo(self.getLocalCRS()))
                sortTrack2DByAngle(self.mws[fIdx],fMotion,np.array([1,0,0]),True)

        #必要なものだけを返す.
        #json.dumpでjson化できる形でなければならないのでTrack3D等はそのままでは返せない.
        #以下のようにメンバ変数を個別に選んでもよいし、nljson型を挟んでもよい.

        #lastTrackInfo = [{'truth':track.truth.__str__(), 'time': track.time, 'pos': list(track.pos), 'vel':list(track.vel)} for track in self.lastTrackInfo]
        #msls = [m() for m in self.msls]
        #return {'ourMotion':self.ourMotion, 'lastTrackInfo': lastTrackInfo, 'msl': msls}

        return nljson({
            'ourMotion':self.ourMotion,
            'lastTrackInfo':self.lastTrackInfo,
            'msls':self.msls
        })() #nljsonを生成してから()でプリミティブ型に変えるのでも可

    def deploy(self,action):
        for pIdx,parent in enumerate(self.parents.values()):
            if(not parent.isAlive()):
                continue

            self.observables[parent.getFullName()]["decision"]={
                "Roll":["Don’t care"],"Horizontal":["Az_BODY",0.0],
                "Vertical":["El",100.0],
                "Throttle":["Vel",150.0],
                "Fire":[False,Track3D().to_json()]
            }

        # observablesから必要な情報を取得
        tick = self.manager.getTickCount() # シミュレーション開始時からの経過tick数を取得
        time = self.manager.getTime() # Timeオブジェクトとしてシミュレーション中の時刻を取得
        elapsedTime = self.manager.getElapsedTime() # シミュレーション開始時からの経過秒数を取得
        dist = {}
        for pIdx, parent in enumerate(self.parents.values()):
            if(parent.isAlive()):
                myMotion=MotionState(parent.observables["motion"]).transformTo(self.getLocalCRS())
                tracks = []
                for t in parent.observables.at_p("/sensor/track"):
                    ret=-1.0
                    track = Track3D(t).transformTo(self.getLocalCRS())
                    tmp=np.linalg.norm(track.pos()-myMotion.pos())
                    if(ret<0 or tmp<ret):
                        ret = tmp
                    tracks.append({'track':track, 'distance':ret})
                dist[pIdx] = tracks

        # actionInfosの更新
        for pIdx,parent in enumerate(self.parents.values()):
            if(not parent.isAlive()):
                continue
            # 自陣営の機体に最も近いものの速さを自分の速さに設定する
            self.actionInfos[pIdx].dstV = action
            
            # 自分に最も近い機体との距離が20000以下の場合は逃げるために距離をとり, それ以外の場合は接近する
            # 何も観測できていない場合は前回の値のまま変更しない
            if len(dist[pIdx]) > 0:
                myMotion=MotionState(parent.observables["motion"]).transformTo(self.getLocalCRS())
                d = sorted(dist[pIdx], key=lambda x: x['distance'])[0]
                diff=myMotion.pos()-d['track'].pos()
                if d['distance'] <= 20000:
                    print('escape')
                    self.actionInfos[pIdx].dstDir=diff/np.linalg.norm(diff)
                else:
                    print('move forward')
                    self.actionInfos[pIdx].dstDir=-diff/np.linalg.norm(diff)

            # 目標進行方向のz成分が正で高度が5000以下の場合は上昇
            if self.actionInfos[pIdx].dstDir[2]>0 and np.array(myMotion.pos())[2]>=-5000:
                print('up')
                self.actionInfos[pIdx].dstDir[2] = -self.actionInfos[pIdx].dstDir[2]

            # 自分に最も近い機体との距離が20000以上22000以下の場合は対象に対して射撃する
            # 何も観測できていない場合は何もしない
            if len(dist[pIdx]) > 0:
                self.actionInfos[pIdx].target = d['track']
                if d['distance'] >= 20000 and d['distance']<=22000:
                    print('launch')
                    self.actionInfos[pIdx].launchFlag=True
                else:
                    self.actionInfos[pIdx].launchFlag=False
            else:
                self.actionInfos[pIdx].launchFlag=False


        # commandsに設定
        # これまでの演算をlocalCRS上で行ってきたが,
        # commandsでparentに渡すときはparentの座標系に戻さなければならない.
        for pIdx,parent in enumerate(self.parents.values()):
            if(not parent.isAlive()):
                continue
            originalMyMotion=MotionState(parent.observables["motion"])
            myMotion=MotionState(parent.observables["motion"]).transformTo(self.getLocalCRS())
            print(pIdx, myMotion.pos(), self.actionInfos[pIdx].dstV, self.actionInfos[pIdx].dstDir)

            #元のparent座標系に戻す
            cmdDstDir=originalMyMotion.dirAtoP(self.actionInfos[pIdx].dstDir,myMotion.pos(),self.getLocalCRS())

            self.commands[parent.getFullName()] = {
                "motion": { #機動の指定。以下の指定方法は一例。
                    "dstDir": cmdDstDir, #進みたい方向を指定
                    "dstV": self.actionInfos[pIdx].dstV #進みたい速度を指定
                },
                "weapon": {
                    "launch": self.actionInfos[pIdx].launchFlag, #射撃要否を bool で指定
                    "target": self.actionInfos[pIdx].target #射撃目標の Track3D を 指定
                }
            }


    def control(self):
        pass

## 投稿可能なプログラム一式としてまとめる

`Agents/MyAgent/__init__.py`を以下のように編集する. 中央集権方式なので, `isUserAgentSingleAsset`メソッドでは`False`を返すようにしている.

In [ ]:
import os,json
import numpy as np
from ASRCAISim1.policy import StandalonePolicy
from ASRCAISim1.core import MotionState

def getUserAgentClass(args={}):
    from .MyAgent import SampleAgent
    return SampleAgent


def getUserAgentModelConfig(args={}):
    configs=json.load(open(os.path.join(os.path.dirname(__file__),"config.json"),"r"))

    return configs


def isUserAgentSingleAsset(args={}):
    return False


class DummyPolicy(StandalonePolicy):
    def step(self,observation,reward,done,info,agentFullName,observation_space,action_space):
        if len(observation['lastTrackInfo']) > 0:
            vel = np.linalg.norm(observation['lastTrackInfo'][0]['vel'])
        else:
            # 何も観測されていない場合は最初のparentの速度を使用
            vel = np.linalg.norm(MotionState(observation['ourMotion'][0]).vel())
        return vel


def getUserPolicy(args={}):
    return DummyPolicy()

`config.json`を以下のように編集して`Agents/MyAgent`以下に保存する(実装したアルゴリズムに応じて適切に作成する想定).

```json
{
    "interval":{
        "unit":"time",
        "step":3.0,
        "perceive":1.0,
        "control":1.0,
        "behave":1.0
    }
}
```

`__init__.py`の各関数に渡す引数として`args.json`を`Agents/MyAgent`以下に保存する(実装したアルゴリズムに応じて適切に作成する想定).

```json
{
    "type": "original"
}
```

`Agents/MyAgent`が以下のようなディレクトリ構造になっていることを確認.

```bash
MyAgent
├── __init__.py
├── args.json
├── config.json
└── MyAgent.py
```

## 対戦を実行する

作成したエージェントを初期行動判断モデルと戦わせる.

In [ ]:
import os

os.chdir('/workspace/')

In [ ]:
! python validate.py --myagent-module-path Agents/MyAgent --num-validation 1

適宜`--color`を"Blue"や"Red"に変えて陣営の種類に応じた行動が取れているかなどを確認する. また, `--movie`と`--visualize`を`1`にして実際にどのように動いているかを確認するなりログを参考にするなりしてアルゴリズムをよりよくする.